In [1]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from bleak import BleakScanner

print('Выберите тип датчика из списка')
#detector_type = str(input('TD, TT, TL, DU, TH, TW, TP: '))

async def run():
    devices = await BleakScanner.discover()  
    for device in devices:
        if devices is not None:
            filtered_devices = [(device.name, device.metadata, device.rssi) for device in devices if device.name is not None and 'TH' in device.name]
            for data in filtered_devices:
                print(data)
                pass
        
        #print(device.name, device.metadata, device.rssi)
        
        if device.name is not None:
            if device.name.startswith('TH'):
                if 'manufacturer_data' in device.metadata:

                    data = device.metadata['manufacturer_data'][3862]

                    print("Device %s (%s), RSSI=%d dB" % (device.address, device.name, device.rssi))

                    # Расшифровка температуры
                    TH09_temp_raw = int.from_bytes(data[1:3], byteorder='little', signed=True)
                    TH09_temp = TH09_temp_raw / 10.0
                    print(f"Температура: {TH09_temp} градусов Цельсия")

                    # Расшифровка освещенности
                    TH09_light_raw = int.from_bytes(data[3:5], byteorder='little', signed=False)
                    print(f"Освещенность: {TH09_light_raw} люкс")

                    # Расшифровка влажности
                    TH09_humidity_raw = int.from_bytes(data[5:7], byteorder='little', signed=False)
                    TH09_humidity = TH09_humidity_raw / 10.0
                    print(f"Влажность: {TH09_humidity}%")
                    
                    # Расшифровка состояния дискретных входов
                    #  Надеюсь сюда тоже идет один байт (не знаю)
                    #discrete_row = int.from_bytes(data[7:8], byteorder='big', signed=False)
                    
                    # Резерв
                    #reserve_row = int.from_bytes(data[8:9], byteorder='big', signed=False)
                    #print('Резерв: не применимо')

                    # Расшифровка режима работы
                    TH09_mode_raw = int.from_bytes(data[9:10], byteorder='big', signed=False)
                    TH09_period = (TH09_mode_raw & 0b00000111) * 20
                    print(f"Период опроса датчиков: {TH09_period} секунд")
                    TH09_flags = TH09_mode_raw & 0b11111000
                    if TH09_flags & 0b10000000:
                        print("Датчик температуры включен")
                    if TH09_flags & 0b01000000:
                        print("Датчик влажности включен")
                    if TH09_flags & 0b00100000:
                        print("Датчик атмосферного давления включен")
                    if TH09_flags & 0b00010000:
                        print("Датчик освещенности включен")
                    if TH09_flags & 0b00001000:
                        print("Датчик холла сработал")

                    # Расшифровка напряжения батареи
                    TH09_battery_raw = int.from_bytes(data[10:11], byteorder='big', signed=False)
                    TH09_battery = TH09_battery_raw / 10.0
                    print(f"Напряжение батареи: {TH09_battery} В")

                    # Расшифровка версии прошивки
                    TH09_version_raw = int.from_bytes(data[11:12], byteorder='big', signed=False)
                    print(f"Версия прошивки: {TH09_version_raw}")
                    print()
        #else:
            #print(type(device.name))
            #print('упало')

loop = asyncio.get_event_loop()
loop.run_until_complete(run())

Выберите тип датчика из списка
